In [ ]:
from langchain_google_cloud_sql_pg import (
    PostgresEngine,
    PostgresSaver,
)
from src.config import env
from langchain_core.runnables import RunnableConfig
from langchain.load import dumpd

from langchain_core.messages import AIMessage, BaseMessage, ToolMessage, HumanMessage

In [ ]:
async def get_ckeckpoint():
    engine = await PostgresEngine.afrom_instance(
        project_id=env.PROJECT_ID,
        region=env.LOCATION,
        instance=env.INSTANCE,
        database=env.DATABASE,
        user=env.DATABASE_USER,
        password=env.DATABASE_PASSWORD,
        engine_args={"pool_pre_ping": True, "pool_recycle": 300},
    )
    checkpointer = await PostgresSaver.create(engine=engine)
    return checkpointer


checkpointer = await get_ckeckpoint()

In [ ]:
user_id = "5521996143632"
# user_id = "5518981524292"
config = RunnableConfig(configurable={"thread_id": user_id})
l = await checkpointer.aget(config=config)
messages = l["channel_values"]["messages"]

In [ ]:
from typing import Sequence, List


def clean_malformed_messages(messages: Sequence[BaseMessage]) -> List[BaseMessage]:
    """
    Remove mensagens mal formadas que podem causar erro no Gemini API.

    Remove AIMessages que têm:
    - content vazio (empty string) E
    - tool_calls vazio (lista vazia ou None)

    Args:
        messages: Sequência de mensagens para limpar

    Returns:
        Lista de mensagens válidas
    """
    cleaned_messages = []

    for message in messages:
        # Verifica se é uma AIMessage com problemas
        if isinstance(message, AIMessage):
            # Verifica se tem conteúdo vazio E tool_calls vazio
            has_empty_content = not message.content or message.content.strip() == ""
            has_empty_tool_calls = (
                not message.tool_calls or len(message.tool_calls) == 0
            )
            finish_reason = message.response_metadata.get("finish_reason")
            # Se ambos estão vazios, pula esta mensagem
            if (
                has_empty_content and has_empty_tool_calls
            ) or finish_reason == "MALFORMED_FUNCTION_CALL":
                print(f"Removendo AIMessage mal formada: ID={message.id}")
                continue

        # Adiciona mensagem válida à lista
        cleaned_messages.append(message)

    return cleaned_messages

In [ ]:
cleaned_messages = clean_malformed_messages(messages=messages)